In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd/content/gdrive/MyDrive/707

/content/gdrive/MyDrive/707


In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=dcb1b046b559cbc0a02b32b6c6a8d9932811634059513e6b3114bcee0a35148e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
ls

'An overview on BigData and Hadoop.pdf'          GDDA707_W9_BigDataPyspark.pdf
 Big_data_analysis_using_Hadoop_cluster.pdf     'GDDA707_Week 9 Lab.ipynb'
 customer_demographics1.csv                     'GGDA707_Lab 11.pdf'
'GDDA707_Assessment 2 V1.1.pdf'                  HDFS/
'GDDA707_Installing Apache Spark and Java.pdf'   sales_transactions.csv
 GDDA707_Lab11_ETL.ipynb


In [ ]:
# Import necessary PySpark libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.types import StringType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ETL Example with PySpark Week 9") \
    .getOrCreate()



In [ ]:
# Step 1: Load Data

sales_df = spark.read.csv("sales_transactions.csv", header=True, inferSchema=True)
customers_df = spark.read.csv("customer_demographics1.csv", header=True, inferSchema=True)

print("All Customers:")
customers_df.show()

duplicate_customers = customers_df.groupBy("customer_id").count().filter("count > 1")
print("Duplicate Customers:")
duplicate_customers.show()

All Customers:
+-----------+----+------+-------------+---------+
|customer_id| age|gender|     location|   income|
+-----------+----+------+-------------+---------+
|        101|  31|  Male|  Los Angeles| 37475.29|
|        102|  60| Other|  Los Angeles| 31138.86|
|        103|  30|  Male|     New York|  68222.3|
|        104|  48|Female|San Francisco| 73901.15|
|        105|  52| Other|        Miami|100590.36|
|        104|  48|Female|San Francisco| 73901.15|
|        106|NULL|Female|San Francisco| 73901.15|
+-----------+----+------+-------------+---------+

Duplicate Customers:
+-----------+-----+
|customer_id|count|
+-----------+-----+
|        104|    2|
+-----------+-----+



In [ ]:
# Step 2: Data Cleaning
# Remove duplicates
sales_df = sales_df.dropDuplicates()
customers_df = customers_df.dropDuplicates()

# Handle missing values
# Handle missing values and drop rows where age is null or zero
customers_df = customers_df.filter((col("age").isNotNull()) & (col("age") != 0))
# Example: Assuming age and income are important fields in customer demographics
customers_df = customers_df.withColumn("age", when(col("age").isNull(), 0).otherwise(col("age")))
customers_df = customers_df.withColumn("income", when(col("income").isNull(), 0).otherwise(col("income")))


In [ ]:

# Step 3: Data Transformation
# Example: Calculate total sales per customer
total_sales_per_customer = sales_df.groupBy("customer_id").sum("amount").withColumnRenamed("sum(amount)", "total_sales")

# Example: Join customer demographics with total sales
customer_sales_data = customers_df.join(total_sales_per_customer, "customer_id", "left")

In [ ]:

# Output transformed data
customer_sales_data.show()

# Stop Spark session
spark.stop()

+-----------+---+------+-------------+---------+------------------+
|customer_id|age|gender|     location|   income|       total_sales|
+-----------+---+------+-------------+---------+------------------+
|        101| 31|  Male|  Los Angeles| 37475.29|4028.1499999999996|
|        102| 60| Other|  Los Angeles| 31138.86| 4910.619999999999|
|        104| 48|Female|San Francisco| 73901.15|           5000.77|
|        103| 30|  Male|     New York|  68222.3|           6214.08|
|        105| 52| Other|        Miami|100590.36|           4560.84|
+-----------+---+------+-------------+---------+------------------+



In [ ]:
import os

java_home = os.environ.get('JAVA_HOME')
if java_home:
    print("JAVA_HOME is set to:", java_home)
else:
    print("JAVA_HOME is not set.")

JAVA_HOME is not set.


In [ ]:
customer_sales_data.show()

Py4JJavaError: An error occurred while calling o73.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnectio